# Projecte github

# Imports

In [ ]:
# Draw the plots immediately after the current cell
%matplotlib inline

import pandas as pd
# Uncomment the following line to install pymysql
#!pip install PyMySQL
import pymysql
import warnings

# POLARS # <- teacher said that this one is 1000% faster than pandas.

# Project configuration

The file that we are going to use to set up the basic structure:

GitHubScraper/JupyterNotebook/Structure/githubProjectStructure.sql

from the github repository.

### Step 1, import the structure
In workbench: 
"Server" -> "Data Import", and then you should select the direction of "githubProjectStructure.sql" using the 3 dots and import from self-contained file like in the following image:

![Javatpoint](./Images/Project_Setup/data_import.png)  

Then scroll down, and create a new Schema by clicking on the "New..." button. Name it: "githubProject".

![Javatpoint](./Images/Project_Setup/create_new_schema.png)  

Scroll to the bottom, and in the bottom right, click on "Start import". If there are no errors, it will transitionate to "Import progress" and indicate success.

Refresh the scehmas:

![Javatpoint](./Images/Project_Setup/refresh.png)  

### Step 2, importing the data from the csv files

Download the csv files [here](https://drive.google.com/drive/folders/1NWhfFss0_M9V_clkcE9TH-Fy3oIEndWV?usp=sharing).

Selecting the "githubProject" scheme, right click and "Table Data Import Wizard", with this, we are going to import each csv file to the matching table.

You have to select the .csv you want to import, for example in this case, the trendVisits.csv

![Javatpoint](./Images/Project_Setup/file_import.png) 

Then, since you already have the structure, you have to use the existing table that matches with the correct csv file:

![Javatpoint](./Images/Project_Setup/use_existing.png) 

After that, check that the variables type are correct and import it. Do this for each csv file.

You can also verify the data by doing this:

![Javatpoint](./Images/Project_Setup/verify.png) 



# Introduction

TODO index or sections indicating what are we doing...

In [ ]:
#https://www.geeksforgeeks.org/connect-to-mysql-using-pymysql-in-python/
# Connect to MySQL database
def tryToConnectMySQL(db_name, db_host, db_port, db_username, db_password):
    try:
        conn = pymysql.connect(host=db_host,
                               port=db_port,
                               user=db_username,
                               password=db_password,
                               db=db_name)
        if conn:
            print("Connection successful")
        else:
            print("Error")
        
        return conn
        
    except pymysql.Error as e:        
        warnings.warn("Error connecting to MySQL:", e)
        return None

# Pass arguments from outside
db_name = "githubProject"
db_host = "localhost"
db_port = 3306
db_username = "root"
db_password = input("Input your password")

dataBaseConnection = tryToConnectMySQL(db_name, db_host, db_port, db_username, db_password)

In [ ]:
# Execute query and return data in pandas dataframe 
def execute_select_query(cursor, query):
    try:
        cursor.execute(query)
        output = cursor.fetchall()
        # Fetch column names from cursor's description
        columns = [desc[0] for desc in cursor.description]
        
        # Convert output to pandas DataFrame
        if output:
            df = pd.DataFrame(output, columns=columns)
            print("Query executed successfully!")
        return output, df
            
    except pymysql.Error as e:
        print("Error executing query:")
        warnings.warn(str(e))
        return None, None

In [ ]:
# Obtain the cursos in order to interact with the DataBase
cursor = dataBaseConnection.cursor()

# Open Source Projects
Repositories with criteria that indicates being not just a public repository, but a project open to contributions.

Ordered by descending contributors count.

In [ ]:
query = """
SELECT r.owner, r.name, r.mainLanguage, MAX(stars) as total_stars, MAX(contributors) as total_contributors, MAX(openPullRequests + closedPullRequests) as total_prs, MAX(openIssues + closedIssues) as total_issues, MAX(watchers) as total_watchers, MAX(stars) as total_stars FROM Repositories r
JOIN RepositoryVisits v
ON r.owner = v.owner AND r.name = v.name
GROUP BY r.owner, r.name, r.mainLanguage
HAVING total_contributors > 5 AND total_issues > 50
ORDER BY total_contributors DESC;
"""

output_repo, df_repo = execute_select_query(cursor, query)
print(len(df_repo), "repositories matching criteria")
df_repo.head(100)
# TODO require lates commit to be recent
# TODO store license as well

# TODO also search for info/resource repositories

The data of these repositories is then extracted to a `.json` for the website to use:

In [ ]:
# Renames or discards languages.
LANGUAGE_REMAP = {
    "TypeScript": "JavaScript",
    "C": "C/C++",
    "C++": "C/C++",
    "Rust": "Others",
    "Ruby": "Others",
    "Go": "Others",
    "Swift": "Others",
    "Clojure": "Others",
    "Haskell": "Others",
    "Vim Script": "",
    "CSS": "",
    "MDX": "",
    "Shell": "",
}

used_languages = set()
used_tags = set()

oss_repos = {}
for index, row in df_repo.iterrows():
    key = row["owner"] + "/" + row["name"]
    main_language = row["mainLanguage"]
    if main_language in LANGUAGE_REMAP:
        main_language = LANGUAGE_REMAP[main_language]
    oss_repos[key] = {
        "topics": set(),
        "languages": set([main_language] if main_language != "" else []),
    }
    used_languages.add(main_language)

# Remaps GitHub topics to the tags that are used in the website.
TAG_MAP = {}
TAG_ALIASES = {
    "Web": ["react", "vue", "web", "reactjs", "css", "chrome-extension", "react-grid", "react-table", "php", "http"],
    "Modding": ["mod", "minecraft", "emulation", "emulator", "forge", "minecraft-launcher", "modrinth", "minecraft-api", "minecraft-server", "bepinex", "unity3d", "unreal", "unity-mono", "craftbukkit", "valheim", "minecraft-mod", "gta5", "fabric"],
    "Data Science": ["math", "numpy", "data-science", "graphql", "data-visualization", "jupyter-notebook"],
    "Machine Learning": ["ml", "pytorch", "deep-learning", "machine-learning", "deep-neural-networks", "tensorflow", "neural-network", "tensor", "computer-vision", "reinforcement-learning", "hyperparameter-tuning", "ai", "artificial-intelligence", "llama", "llms", "llm"],
    "Tool": ["containers", "zsh", "docker", "github", "cli", "searchengine", "postgrest", "devtool", "cloudstorage", "git", "npm", "database", "postgresql", "backend", "shell-scripting", "websocket", "collaboration"],
    "App": ["note-taking", "productivity", "prest", "download", "latex", "text-editor", "curl", "ftp", "bot", "synchronization", "sqlite", "mattermost", "messaging", "conferencing", "remote-desktop"],

    "Resource": ["learn-to-code", "freecodecamp", "curriculum", "certification", "learnopengl", "lists", "resources", "resource", "dataset", "public-api", "public-apis"],
}
# Tags manually added to some repositories (which otherwise lack descriptive ones)
MANUAL_TAGS = {
    "minio/minio": ["Machine Learning"],
    "Aliucord/Aliucord": ["Modding"],
    "cli-guidelines/cli-guidelines": ["Resource"],
    "yjs/yjs": ["Tool"],
    "TigerVNC/tigervnc": ["Tool"],
    "ollama/ollama": ["App"],
}
for tag,aliases in TAG_ALIASES.items():
    for alias in aliases:
        TAG_MAP[alias] = tag

from collections import defaultdict
IGNORED_TAGS = defaultdict(int)
import json
with open("../persistence.json", "r") as f:
    repos_data = json.load(f)["repositories"]

    for key,repo in oss_repos.items():
        repo["description"] = repos_data[key]["description"]
        for tag in repos_data[key]["tags"]:
            if tag in TAG_MAP:
                repo["topics"].add(TAG_MAP[tag])
            else:
                IGNORED_TAGS[tag] += 1
        if key in MANUAL_TAGS:
            for tag in MANUAL_TAGS[key]:
                repo["topics"].add(tag)

        for tag in repo["topics"]:
            used_tags.add(tag)

# Exclude mirrors and other projects that are not contributable projects or unsuitable
del oss_repos["gitlabhq/gitlabhq"]
del oss_repos["qemu/qemu"]
del oss_repos["xasset/xasset"] # Not english.
del oss_repos["jynew/jynew"] # Unity RPG game framework, documentation in chinese-only though.

for key,repo in oss_repos.items():
    if len(repo["topics"]) == 0:
        print("Repo without tags", f"https://github.com/{key}")

# List unused Github topics
# top_ignored_tags = [(tag, count) for tag, count in IGNORED_TAGS.items()]
# top_ignored_tags = sorted(top_ignored_tags, key=lambda x: x[1], reverse=True)
# print(len(top_ignored_tags))
# print(top_ignored_tags)

# Convert sets to lists for json serialization, and add other
# keys the site expects
for key,repo in oss_repos.items():
    repo["owner"] = key.split("/")[0]
    repo["repo"] = key.split("/")[1]
    repo["topics"] = list(repo["topics"])
    repo["languages"] = list(repo["languages"])

with open("repositories.json", "w") as f:
    json.dump(oss_repos, f, indent=2)

print("Valid repositories:", len(oss_repos))
print("Languages used:", used_languages)
print("Tags used:", used_tags)

# Tables
Shows the table for the different entities of the Data Base.

## Repositories

In [ ]:
query = "SELECT * FROM Repositories;"

output_repo, df_repo = execute_select_query(cursor, query)
df_repo.tail(5)

In [ ]:
query = "SELECT * FROM RepositoryVisits;"

output_repo_visists, df_repo_visists = execute_select_query(cursor, query)
df_repo_visists.head(5)

In [ ]:
query = "SELECT * FROM RepositoryTopics;"

output_repo_topics, df_repo_topics = execute_select_query(cursor, query)
df_repo_topics.tail(5)

## Owners

In [ ]:
query = "SELECT * FROM Owners;"
# Use _ to ignore
_, df_owners = execute_select_query(cursor, query)
df_owners.head(5)

In [ ]:
query = "SELECT * FROM OwnerVisits;"

output_owner_visits, df_owner_visits = execute_select_query(cursor, query)
df_owner_visits.head(5)

## Commits

In [ ]:
query = "SELECT * FROM Commits;"

output_commits, df_commits = execute_select_query(cursor, query)
df_commits.head(5)

## Topics

In [ ]:
query = "SELECT * FROM Topics;"

output_topics, df_topics = execute_select_query(cursor, query)
df_topics.tail(5)

In [ ]:
query = "SELECT * FROM TopicVisits;"

_, df_topic_visits = execute_select_query(cursor, query)
df_topic_visits.head(5)

## Trend

In [ ]:
query = "SELECT * FROM TrendVisits;"

_, df_trend_visits = execute_select_query(cursor, query)
df_trend_visits.head(5)

In [ ]:
# Closes the connection
dataBaseConnection.close()

# Analysis

### Basic statistics
* Distribution of main topics (bar or pie chart). Aka, in what proportion of our studied repos are they treated.
* Distribution of languages (bar or pie chart). Like above.
* Time evolution of interest in topics: repositories per topic, followers per topic.

### Dimensional reduction for an overview on repositories
What we mean here is to perform a PCA (principal components analysis) in order to able to have an insight on the structre of the whole dataset. We would then color the data points in it depending on the language used, for instance, to see if these groups have similar characteristics and lie close in the dataframe or not.
* Create a dataframe containing all RepositoryVisits data for a certain date for all the studied repositories.
* Perform a standarization on the data (i.e., to prevent some variables such as commits to be far more important than others such as forks).
* Perform a PCA into 2 components on it.
* Plot the results while clustering the points depending on different criteria:
    + mainLanguage
    + topic
* Supervised machine learning using stars.
### Open questions
* How to use stars and trends?
* How to use contributions by owners?


# Commits analisis

In [ ]:
import numpy as np
from os import path
from PIL import Image
"""
If 
!pip install wordcloud 
doesn't work:
import sys
print(sys.executable) # use the path

path -m pip install wordcloud
"""
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
# https://www.datacamp.com/tutorial/wordcloud-python

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def plot_word_cloud(word_cloud, text, save_image = False, image_name = "none"):
    # Create and generate a word cloud image:
    word_cloud_output = word_cloud.generate(text)
    
    # Display the generated image:
    plt.imshow(word_cloud_output, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    if save_image:
        word_cloud_output.to_file(f"./Images/wordclouds/{image_name}_word_cloud.png")
        
def transform_format(val):
    if val == 0:
        return 255
    else:
        return val
        
def obtain_mask(mask):    
    trans_mask = np.ndarray((mask.shape[0],mask.shape[1]), np.int32)
    for i in range(len(mask)):
        trans_mask[i] = list(map(transform_format, mask[i]))
    return trans_mask        

## Data cleaning

### Remove stop words

In [ ]:
stopwords = set(STOPWORDS)
# stopwords.update(["a"]) # manually add stopwords

### Remove commits made by bots

In [ ]:
# Find all the commits authored by a bot.
from IPython.display import display
pd.set_option('display.max_rows', None)

cond_a = df_commits[df_commits["author"].str.contains("\[bot]|-bot")]
cond_c = df_commits[df_commits["message"].str.contains("dependabot")]
cond_d = df_commits[df_commits["message"].str.contains("renovatebot")]

temp = pd.concat([cond_a])

unique_auth = pd.unique(temp.author)
unique_msg = pd.unique(cond_c.message)

dict = {'bots' : unique_auth}
df_bots = pd.DataFrame(dict)

# displaying the bots names
display(df_bots)

dict = {'bots_message' : unique_msg}
df_bots_msg = pd.DataFrame(dict)

display(df_bots_msg)

In [ ]:
# Filtering the bots
print("With bots", len(df_commits))
df_commits = df_commits[~df_commits["author"].isin(df_bots["bots"])]
print("Without bots", len(df_commits))

In [ ]:
df_commits_small = df_commits.tail() # only last 5
print(df_commits_small.tail().message, "\n")
small_text_sample = " ".join(commit for commit in df_commits_small.message) # concatenate them
print("Concatenated text:", text)
image_name = "first_word_cloud"
word_cloud = WordCloud(stopwords=stopwords, max_font_size=50, max_words=100, background_color="black")

In [ ]:
text_hyper_small = df_commits["message"][0]

image_name = "first_word_cloud"
word_cloud = WordCloud(stopwords=stopwords, max_font_size=50, max_words=100, background_color="black")

plot_word_cloud(word_cloud, text_hyper_small)

In [ ]:
all_text = " ".join(commit for commit in df_commits.message) # concatenate them
word_cloud = WordCloud(stopwords=stopwords, max_font_size=30, max_words=200, background_color="black")
plot_word_cloud(word_cloud, all_text)

In [ ]:
# Use a mask and a color map
        
# <a href='https://dryicons.com/icon/square-github-icon-8312'> Icon by Dryicons </a>
github_image = np.array(Image.open("./Images/wordclouds/github_square.png"))

word_cloud_mask = obtain_mask(github_image)
# Word cloud repeats words since he needs fill the gaps using the correct size.
# print(len(all_text.split(' ')))
# print(len(pd.unique(all_text.split(' '))))

word_cloud = WordCloud(stopwords=stopwords, colormap='rainbow', mask=word_cloud_mask, max_font_size=30, max_words=10000, background_color="#31003C")
plot_word_cloud(word_cloud, all_text, True, "git_mask")

In [ ]:
# Use a mask        
git_image = np.array(Image.open("./Images/wordclouds/git.png"))

word_cloud_mask = obtain_mask(git_image)

word_cloud = WordCloud(stopwords=stopwords, mask=word_cloud_mask,colormap='hot', max_font_size=20, max_words=10000, background_color="#474747")
plot_word_cloud(word_cloud, all_text)

In [ ]:
# Here goes some grafics bars

In [ ]:
#Could explore the wordcloud of specific projects too:
#TODO